In [ ]:
import os
import re
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
from collections import defaultdict
from tqdm import tqdm

In [ ]:
DATA_DIR = "IXI HQ NII Data Directory"
OUTPUT_DIR = "IXI LQ VOLUMES Output Directory"

target_shape = (179, 221, 200)
target_spacing = (1.0, 1.0, 1.0)

os.makedirs(OUTPUT_DIR, exist_ok=True)

patient_slices = defaultdict(list)

for fname in os.listdir(DATA_DIR):
    if not fname.endswith(".nii.gz"):
        continue
    # IXI002-Guys-0828-T2_0.nii.gz → IXI002-Guys-0828-T2
    base = re.sub(r'_\d+\.nii\.gz$', '', fname)
    patient_slices[base].append(fname)

print(f"Found {len(patient_slices)} patients")
for patient, files in tqdm(patient_slices.items()):
    files = sorted(files, key=lambda x: int(re.search(r'_(\d+)\.nii\.gz', x).group(1)))
    slices = []
    affine = None
    for f in files:
        path = os.path.join(DATA_DIR, f)
        img = nib.load(path)
        data = img.get_fdata()
        if data.ndim == 3 and data.shape[2] == 1:
            data = data[:, :, 0]

        slices.append(data)

        if affine is None:
            affine = img.affine
    volume = np.stack(slices, axis=-1)  # (H, W, D)
    zoom_factors = [
        target_shape[0] / volume.shape[0],
        target_shape[1] / volume.shape[1],
        target_shape[2] / volume.shape[2]
    ]

    volume_resampled = zoom(volume, zoom_factors, order=3)  # cubic
    new_affine = np.diag([1.0, 1.0, 1.0, 1.0])
    out_path = os.path.join(OUTPUT_DIR, f"{patient}.nii.gz")
    new_img = nib.Nifti1Image(volume_resampled.astype(np.float32), new_affine)
    nib.save(new_img, out_path)

print("HQ conversion complete.")

In [ ]:
DATA_DIR = "IXI LQ NII Data Directory"
OUTPUT_DIR = "IXI LQ VOLUMES Output Directory"

target_shape = (112, 138, 40)        # (H, W, D)
target_spacing = (1.6, 1.6, 5.0)     # voxel size in mm

os.makedirs(OUTPUT_DIR, exist_ok=True)
patient_slices = defaultdict(list)

for fname in os.listdir(DATA_DIR):
    if not fname.endswith(".nii.gz"):
        continue

    base = re.sub(r'_\d+\.nii\.gz$', '', fname)
    patient_slices[base].append(fname)

print(f"Found {len(patient_slices)} patients")
for patient, files in tqdm(patient_slices.items()):
    files = sorted(files, key=lambda x: int(re.search(r'_(\d+)\.nii\.gz', x).group(1)))
    slices = []
    affine = None

    for f in files:
        path = os.path.join(DATA_DIR, f)
        img = nib.load(path)
        data = img.get_fdata()
        if data.ndim == 3 and data.shape[2] == 1:
            data = data[:, :, 0]

        slices.append(data)

        if affine is None:
            affine = img.affine
    volume = np.stack(slices, axis=-1)  # (H, W, D)
    zoom_factors = [
        target_shape[0] / volume.shape[0],
        target_shape[1] / volume.shape[1],
        target_shape[2] / volume.shape[2]
    ]

    volume_resampled = zoom(volume, zoom_factors, order=3)  # cubic interpolation
    new_affine = np.diag([
        target_spacing[0],
        target_spacing[1],
        target_spacing[2],
        1.0
    ])
    out_path = os.path.join(OUTPUT_DIR, f"{patient}.nii.gz")
    new_img = nib.Nifti1Image(volume_resampled.astype(np.float32), new_affine)
    nib.save(new_img, out_path)

print("LQ conversion complete.")